INLINE CSV

In [0]:
csv_data ="""transaction_id,customer_name,region,product,category,quantity,unit_price,date
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31
7,Aman,South,TV,Electronics,1,45000,2024-02-15
8,Isha,North,Notebook,Stationery,10,60,2024-01-10
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19
"""
with open("/tmp/sales_transactions.csv", "w") as f:
  dbutils.fs.put("dbfs:/tmp/sales_transactions.csv", csv_data, overwrite=True)


Wrote 603 bytes.


# TASK 1

Load the CSV into a PySpark DataFrame.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("transactions").getOrCreate()

In [0]:
df = spark.read.option("header",True).option("inferSchema",True).csv("dbfs:/tmp/sales_transactions.csv")
df.show()

+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|
|             4|        Divya| North|         Mobile|  Electronics|       2|     20000|2024-03-22|
|             5|       Vikram|  East|Washing Machine|  Electronics|       1|     28000|2024-02-28|
|             6|       Preeti|  West|       Sneakers|      Fashion|       2|      4000|2024-01-31|
|             7|         Aman| South|             TV|  Electronics|       1|     45000|2024-02-15|
|         

Save it in Parquet and Delta formats (in different folders).

In [0]:
df.write.mode("overwrite").parquet("dbfs:/tmp/sales_transactions.parquet")

In [0]:
df.write.format("delta").mode("overwrite").save("dbfs:/tmp/sales_transactions.delta")

Create and register Delta Table called sales_transactions

In [0]:
df_delta = spark.read.format("delta").load("dbfs:/tmp/sales_transactions.delta")
df_delta.show()

+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|
|             4|        Divya| North|         Mobile|  Electronics|       2|     20000|2024-03-22|
|             5|       Vikram|  East|Washing Machine|  Electronics|       1|     28000|2024-02-28|
|             6|       Preeti|  West|       Sneakers|      Fashion|       2|      4000|2024-01-31|
|             7|         Aman| South|             TV|  Electronics|       1|     45000|2024-02-15|
|         

# TASK 2


Add a new column total_amount = quantity × unit_price .

In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forPath(spark, "dbfs:/tmp/sales_transactions.delta")
delta_table.toDF().show()

+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|
|             4|        Divya| North|         Mobile|  Electronics|       2|     20000|2024-03-22|
|             5|       Vikram|  East|Washing Machine|  Electronics|       1|     28000|2024-02-28|
|             6|       Preeti|  West|       Sneakers|      Fashion|       2|      4000|2024-01-31|
|             7|         Aman| South|             TV|  Electronics|       1|     45000|2024-02-15|
|         

In [0]:
from pyspark.sql.functions import col,month,date_format
delta = delta_table.toDF()
delta_path = "dbfs:/tmp/delta/sales_transactions"
df_transformed = (df
                  .withColumn("total_amount", col("quantity")*col("unit_price"))
                  .withColumn("month",month(col("date")))
                  .withColumn("date_formattedd",date_format(col("date"),"dd-MM-yyyy"))
                  .withColumn("is_high_value",col("total_amount")>30000))
df_transformed.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_path)

In [0]:
delta = DeltaTable.forPath(spark, delta_path)
delta.toDF().show()

+--------------+-------------+------+---------------+-------------+--------+----------+----------+------------+-----+---------------+-------------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|total_amount|month|date_formattedd|is_high_value|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+------------+-----+---------------+-------------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|       55000|    1|     12-01-2024|         true|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|       32000|    2|     05-02-2024|         true|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|         750|    1|     17-01-2024|        false|
|             4|        Divya| North|         Mobile|  Electronics|       2|     20000|2024-03-22|       40000| 

# TASK 3

In [0]:
df = DeltaTable.forPath(spark,delta_path).toDF()


In [0]:
from pyspark.sql.functions import count, sum, col, desc
transactions = df.groupBy("region").agg(count("*").alias("transactions_count"))
print("TRANSACTIONS COUNT")
transactions.show()

top_cate = df.groupBy("category").agg(sum("total_amount").alias("total_sales")).orderBy(col("total_sales").desc()).limit(3)
print("TOP 3 CATEGORIES")
top_cate.show()

month_wise = df.groupBy("month").agg(sum("total_amount").alias("total_sales")).orderBy(col("month"))
print("MONTHLY SALES")
month_wise.show()

highest = df.orderBy(desc("total_amount")).limit(1)
print("HIGHEST SALES")
highest.show()

q1 = df.filter(col("month").isin([1,2,3])).agg(sum("total_amount").alias("q1 total_sales"))
print("Q1 SALES")
q1.show()

TRANSACTIONS COUNT
+------+------------------+
|region|transactions_count|
+------+------------------+
| South|                 2|
|  East|                 2|
|  West|                 3|
| North|                 3|
+------+------------------+

TOP 3 CATEGORIES
+-------------+-----------+
|     category|total_sales|
+-------------+-----------+
|  Electronics|     200000|
|      Fashion|       8000|
|Personal Care|       1350|
+-------------+-----------+

MONTHLY SALES
+-----+-----------+
|month|total_sales|
+-----+-----------+
|    1|      64350|
|    2|     105000|
|    3|      40800|
+-----+-----------+

HIGHEST SALES
+--------------+-------------+------+-------+-----------+--------+----------+----------+------------+-----+---------------+-------------+
|transaction_id|customer_name|region|product|   category|quantity|unit_price|      date|total_amount|month|date_formattedd|is_high_value|
+--------------+-------------+------+-------+-----------+--------+----------+----------+---------

# TASK 4

In [0]:
delta_table.toDF().show()

+--------------+-------------+------+---------------+-------------+--------+----------+----------+------------+-----+---------------+-------------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|total_amount|month|date_formattedd|is_high_value|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+------------+-----+---------------+-------------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|       55000|    1|     12-01-2024|         true|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|       32000|    2|     05-02-2024|         true|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|         750|    1|     17-01-2024|        false|
|             4|        Divya| North|         Mobile|  Electronics|       2|     20000|2024-03-22|       40000| 

In [0]:
delta_table.update(
    condition=col("category") == "Stationery",
    set={"unit_price": col("unit_price") * 1.10}
)

print("UPDATED-10% increase for Stationery")
delta_table.toDF().show()

delta_table.delete(
    condition=col("quantity") < 3
)

print("AFTER DELETIONv -  quantity < 3 removed")
delta_table.toDF().show()


today_data = [
    (11, "Meera", "North", "Smartwatch", "Electronics", 1, 1500, date.today(), 1500, 8, date.today().strftime("%d-%m-%Y"), False)
]

columns = [
    "transaction_id", "customer_name", "region", "product", "category",
    "quantity", "unit_price", "date", "total_amount", "month",
    "date_formattedd", "is_high_value"
]

new_df = spark.createDataFrame(today_data, columns)

delta_table.alias("target") \
    .merge(
        new_df.alias("source"),
        "target.transaction_id = source.transaction_id"
    ) \
    .whenNotMatchedInsertAll() \
    .execute()

print("AFTER MERGE- new transaction inserted ")
delta_table.toDF().show()

UPDATED-10% increase for Stationery
+--------------+-------------+------+---------------+-------------+--------+----------+----------+------------+-----+---------------+-------------+
|transaction_id|customer_name|region|        product|     category|quantity|unit_price|      date|total_amount|month|date_formattedd|is_high_value|
+--------------+-------------+------+---------------+-------------+--------+----------+----------+------------+-----+---------------+-------------+
|             1|       Rajesh| North|         Laptop|  Electronics|       1|     55000|2024-01-12|       55000|    1|     12-01-2024|         true|
|             2|        Sneha|  West|   Refrigerator|  Electronics|       1|     32000|2024-02-05|       32000|    2|     05-02-2024|         true|
|             3|         Anil| South|        Shampoo|Personal Care|       5|       150|2024-01-17|         750|    1|     17-01-2024|        false|
|             4|        Divya| North|         Mobile|  Electronics|       2|

# TASK 5

In [0]:
partitioned_by_region_path = "/tmp/delta/sales_transactions_by_region"

delta_table = DeltaTable.forPath(spark, delta_path)
delta_table.toDF() \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("region") \
    .save(partitioned_by_region_path)

print("Delta table rewritten partitioned by region.")

partitioned_by_month_path = "/tmp/delta/sales_transactions_by_month"

delta_table.toDF() \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("month") \
    .save(partitioned_by_month_path)

print("Second Delta table created partitioned by month.")

spark.sql(f"""
OPTIMIZE delta.`{partitioned_by_region_path}`
ZORDER BY (category)
""")

spark.sql(f"""
OPTIMIZE delta.`{partitioned_by_month_path}`
ZORDER BY (category)
""")

print("Both Delta tables optimized with Z-Ordering on 'category'.")

Delta table rewritten partitioned by region.
Second Delta table created partitioned by month.
Both Delta tables optimized with Z-Ordering on 'category'.
